In [8]:
!pip install -q -U langchain langchain-groq duckduckgo-search

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\ddgs.exe' -> 'C:\\Python311\\Scripts\\ddgs.exe.deleteme'



In [9]:
with open("groq.txt", 'r') as f:
    key = f.read().strip()

In [10]:
from langchain_groq import ChatGroq
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import pandas as pd
import requests
import calendar
from typing import List

In [33]:
chat = ChatGroq(temperature=0, groq_api_key=key, model_name="llama3-70b-8192")

In [34]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchRun()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [35]:
def convert_train_info_to_string(info):

  return (
      f"Train {info['name']} (number {info['number']}) arrives at {info['arrival time']}"
      f" and reaches {info['reaching time']}. Travel time is {info['duration (in hour)']} hours."
      f" Runs on {info['running days']}.")

In [41]:
def TrainBetweenStation(input_str : str, pref_running_days: List[str]=[]):
    source = input_str.split()[0]
    destination = input_str.split()[1]
    data = pd.read_csv("Rail_data.txt", low_memory=False)
    Station_Code = data[["Station Name", "Station Code"]]
    Station_Code = Station_Code.drop_duplicates(subset=['Station Name'])
    Station_Code["Station Name"] = Station_Code["Station Name"].apply(lambda x: x.split(" ")[0])
    Station_Code = Station_Code.reset_index()
    scode = Station_Code.loc[Station_Code['Station Name'] == source.upper(), 'Station Code'].iloc[0]
    dcode = Station_Code.loc[Station_Code['Station Name'] == destination.upper(), 'Station Code'].iloc[0]

    url = f"https://indian-railway-api.cyclic.app/trains/betweenStations/?from={scode}&to={dcode}"

    payload={}
    headers = {}
    output = ""
    
    response = requests.request("GET", url, headers=headers, data=payload)

    for i, res in enumerate(response.json()["data"]):
        name = res["train_base"]["train_name"]
        number = res["train_base"]["train_no"]
        ftime = res["train_base"]["from_time"]
        ttime = res["train_base"]["to_time"]
        trtime = res["train_base"]["travel_time"]
        rdays = res["train_base"]["running_days"]
        
        days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        running_days = []
        
        for j, char in enumerate(rdays):
            if char == '1':
                running_days.append(calendar.day_name[j])
        rdays = ', '.join(running_days)

        if float(ftime[:2])>=12:
            ftime+=" PM"
        else:
            ftime+=" AM"

        if float(ttime[:2])>=12:
            ttime+=" PM"
        else:
            ttime+=" AM"

        if len(pref_running_days) == 0:
            print("Train No. :", number)
            print("Train Name :", name)
            print("Arrival Time :", ftime)
            print("Reaching Time :", ttime)
            print("Duration (in hours) :", trtime)
            print("Running Days :", rdays)
            print("********************")
            info = {"name":name,
                    "number":number,
                    "arrival time":ftime,
                    "reaching time":ttime,
                    "duration (in hour)":trtime,
                    "running days":rdays
                   }
            output += convert_train_info_to_string(info)
            output += "\n********************\n"
        else:
            search_set = set(pref_running_days)
            all_found = search_set.intersection(set(running_days))
            if all_found:  
                print("Train No. :", number)
                print("Train Name :", name)
                print("Arrival Time :", ftime)
                print("Reaching Time :", ttime)
                print("Duration (in hours) :", trtime)
                print("Running Days :", rdays)
                print("********************")
                info = {"name":name,
                    "number":number,
                    "arrival time":ftime,
                    "reaching time":ttime,
                    "duration (in hour)":trtime,
                    "running days":rdays
                   }
                output += convert_train_info_to_string(info)
                output += "\n********************\n"
                

    return output

In [42]:
TrainBetweenStation = Tool(
    name='TrainBetweenStation',
    func= TrainBetweenStation,
    description="Use this tool when user ask to find trains between source station to destination station. this tool accepts 1 argument, source and destination station name."
)

In [43]:
from langchain.agents import initialize_agent

tools = [search, TrainBetweenStation]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=chat,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [44]:
conversational_agent("Get me all train from kharagpur to kamakhya and display the result")



> Entering new AgentExecutor chain...
```
{
    "action": "TrainBetweenStation",
    "action_input": "kharagpur kamakhya"
}
```Train No. : 12507
Train Name : ARONAI EXPRESS
Arrival Time : 09.13 AM
Reaching Time : 04.40 AM
Duration (in hours) : 19.27
Running Days : Thursday
********************
Train No. : 12509
Train Name : GUWAHATI EXP
Arrival Time : 07.05 AM
Reaching Time : 04.10 AM
Duration (in hours) : 21.05
Running Days : Friday, Saturday, Sunday
********************
Train No. : 12503
Train Name : AGTL HUMSAFAR
Arrival Time : 16.14 PM
Reaching Time : 12.15 PM
Duration (in hours) : 20.01
Running Days : Wednesday, Saturday
********************
Train No. : 12513
Train Name : SC SCL SF EXP
Arrival Time : 16.15 PM
Reaching Time : 11.55 AM
Duration (in hours) : 19.40
Running Days : Sunday
********************
Train No. : 12515
Train Name : CBE SCL SF EXP
Arrival Time : 07.05 AM
Reaching Time : 04.10 AM
Duration (in hours) : 21.05
Running Days : Tuesday
********************
Train No. :

{'input': 'Get me all train from kharagpur to kamakhya and display the result',
 'chat_history': [],
 'output': 'Multiple trains are available from Kharagpur to Kamakhya. Please find the details above.'}